In [2]:
!pip install geopy

Defaulting to user installation because normal site-packages is not writeable
  Using cached geopy-2.3.0-py3-none-any.whl (119 kB)
  Using cached geographiclib-2.0-py3-none-any.whl (40 kB)


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from geopy.geocoders import Photon
from geopy.extra.rate_limiter import RateLimiter
from time import sleep
from geopy.exc import GeocoderUnavailable

In [2]:
spark = SparkSession.builder.appName("get uf").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/06 07:31:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:

def get_uf(cidade):
    states = {'Acre','Alagoas','Amapá','Amazonas','Bahia','Ceará','Distrito Federal',
            'Espírito Santo','Goiás','Maranhão','Mato Grosso','Mato Grosso do Sul',
            'Minas Gerais','Pará','Paraíba','Paraná','Pernambuco','Piauí',
            'Rio de Janeiro','Rio Grande do Norte','Rio Grande do Sul','Rondônia',
            'Roraima','Santa Catarina','São Paulo','Sergipe','Tocantins'
            }
    sleep(1.1)
    attempt=1
    max_attempts=5
    try:
        geolocator = Photon()
        locate = RateLimiter(geolocator.geocode, min_delay_seconds=1)
        adress = locate(cidade)
        uf = set(adress[:-1][0].split(', '))
        if states.intersection(uf):
            attempt=0
            return list(states.intersection(uf))[0]
        else:
            attempt=0
            return 'erro'
    except GeocoderUnavailable:
        if attempt <= max_attempts:
            attempt+=1
            return get_uf(cidade)
        raise


In [4]:
dados = spark.read.json('/home/guilherme.cunha/Área de Trabalho/WORKSPACE/Compass/Testes/a', multiLine=True)

In [5]:
def analyser(text):
    import requests
    import nltk
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    nltk.download("vader_lexicon")
    link = 'https://raw.githubusercontent.com/rafjaa/LeIA/master/lexicons/vader_lexicon_ptbr.txt'
    req = requests.get(link)
    req = req.text
    lista = req.split('\n')
    lista.pop()
    dicionario = {}
    for linha in lista:
        linha = linha.split('\t')
        dicionario[linha[0]]= float(linha[1])
    sent_analyzer = SentimentIntensityAnalyzer()
    sent_analyzer.lexicon.update(dicionario)
    result_dict = sent_analyzer.polarity_scores(text)
    # result = 'positivo' if result_dict['compound'] > 0 else('negativo' if result_dict['compound'] < 0 else 'neutro')
    return result_dict['compound']

In [6]:
analyze = udf(analyser, FloatType())
getuf = udf(get_uf, StringType())

In [7]:
dados = dados.select('*').withColumn(
        'tweet_text',lower('tweet_text')
).withColumn(
    'sentimento', when(
        analyze('tweet_text') > 0 , 'positivo'
    ).when(
        analyze('tweet_text') < 0 , 'negativo'
    ).when(
        analyze('tweet_text') == 0 , 'neutro'
    )
).withColumn('pontuacao',
    when(
       col('sentimento') == 'neutro', '0' 
    ).otherwise(
        analyze('tweet_text')
    )
).withColumn(
    'cidade', split(col('geo'),', ')[0]
).withColumn(
    'pais', split(col('geo'),', ')[1])

In [8]:
dados = dados.where(col('pais').isNotNull())

In [9]:
dados = dados.withColumn("year", date_format(col("tweet_date"), "yyyy")) \
            .withColumn("month", date_format(col("tweet_date"), "MM")) \
            .withColumn("day", date_format(col("tweet_date"), "dd")) \
            .withColumn('tweet_text', regexp_replace('tweet_text','\n',' '))

In [ ]:
dados.show()

+---------+--------------------+-------------------+-------------------+--------------------+----------+---------+--------------------+------+----+-----+---+
|candidato|                 geo|                 id|         tweet_date|          tweet_text|sentimento|pontuacao|              cidade|  pais|year|month|day|
+---------+--------------------+-------------------+-------------------+--------------------+----------+---------+--------------------+------+----+-----+---+
|Bolsonaro|   Sao Paulo, Brazil|1597943225108418560|2022-11-30 13:18:47|@belloniviviane @...|  negativo|  -0.6808|           Sao Paulo|Brazil|2022|   11| 30|
|Bolsonaro|    Brasília, Brazil|1597943253797867522|2022-11-30 13:18:54|@vanessnnavarro @...|  positivo|   0.6369|            Brasília|Brazil|2022|   11| 30|
|Bolsonaro|Rio de Janeiro, B...|1597943318390132736|2022-11-30 13:19:10|@folha dispara na...|    neutro|        0|      Rio de Janeiro|Brazil|2022|   11| 30|
|Bolsonaro|São Leopoldo, Brasil|1597943333078593537|

In [10]:
dados = dados.where((dados.pais == 'Brasil') | (dados.pais == 'Brazil'))

In [ ]:
dados.groupBy('pais').count().sort('pais').show()

+------+-----+
|  pais|count|
+------+-----+
|Brasil| 1721|
|Brazil| 1905|
+------+-----+



In [11]:
dados = dados.withColumn('uf', getuf(col('geo')))

In [ ]:
dados.write.mode("append").partitionBy("year","uf",'candidato') \
    .parquet('/home/guilhermefmk/Área de Trabalho/WORKSPACE/Compass/Testes/ufs')

In [ ]:
dados.groupBy(['uf','candidato','sentimento']).count().sort('uf').show()

In [21]:
final = spark.read.parquet('/home/guilherme.cunha/Área de Trabalho/WORKSPACE/Compass/Testes/ufs')

In [ ]:
def format_date(date):
    if date.find(':') != -1:
        result = datetime.strftime(datetime.strptime(date,'%Y-%m-%d %H:%M:%S'), '%Y-%m-%d %H:%M:%S')
    else:
        result = datetime.strftime(datetime.strptime(date,'%Y-%m-%d %H-%M-%S'), '%Y-%m-%d %H:%M:%S')
    return result

In [ ]:
format_date('2022-12-02 16-18-48')

'2022-12-02 16:18:48'

In [ ]:
from datetime import datetime

In [ ]:
getdate = udf(format_date, StringType())

In [ ]:
dados = dados.withColumn('tweet_date', getdate('tweet_date'))

In [ ]:
dados = dados.withColumn("tweet_date", to_timestamp(getdate('tweet_date')))

In [ ]:
dados.printSchema()

root
 |-- candidato: string (nullable = true)
 |-- geo: string (nullable = true)
 |-- id: string (nullable = true)
 |-- tweet_date: timestamp (nullable = true)
 |-- tweet_text: string (nullable = true)



In [ ]:
dados.where(dados.tweet_date.isNull()).show()

+---------+---+---+----------+----------+
|candidato|geo| id|tweet_date|tweet_text|
+---------+---+---+----------+----------+
+---------+---+---+----------+----------+



In [22]:
final = final.withColumn('pontuacao', col("pontuacao").cast(FloatType()))

In [24]:
final.where(final.uf == 'erro').count()

38

In [25]:
final = final.where(final.uf != 'erro')

In [36]:
final = final.groupBy(['uf','candidato','sentimento']).agg(count('sentimento').alias('contador'),avg('pontuacao'))

In [39]:
final = final.withColumn('year', lit(2022))

In [41]:

final.write.mode("append").partitionBy("year","candidato","uf",'sentimento') \
    .parquet('/home/guilherme.cunha/Área de Trabalho/WORKSPACE/Compass/Testes/teste')

In [42]:
sad = spark.read.parquet('/home/guilherme.cunha/Área de Trabalho/WORKSPACE/Compass/Testes/teste')

In [43]:
sad.show()

+--------+--------------------+----+---------+-----------------+----------+
|contador|      avg(pontuacao)|year|candidato|               uf|sentimento|
+--------+--------------------+----+---------+-----------------+----------+
|       5| 0.47173999547958373|2022|Bolsonaro|   Espírito Santo|  positivo|
|      15|-0.47188666462898254|2022|Bolsonaro|          Paraíba|  negativo|
|      18|                 0.0|2022|Bolsonaro|          Paraíba|    neutro|
|       3|                 0.0|2022|     Lula|         Maranhão|    neutro|
|      11|                 0.0|2022|     Lula|      Mato Grosso|    neutro|
|       6|-0.43701666096846264|2022|     Lula|         Maranhão|  negativo|
|       1|-0.42149999737739563|2022|     Lula|        Tocantins|  negativo|
|       1|-0.15309999883174896|2022|     Lula|             Acre|  negativo|
|      16|                 0.0|2022|     Lula|            Ceará|    neutro|
|       7|-0.34401429125240873|2022|     Lula|            Goiás|  negativo|
|       3|-0

In [ ]:
teste = spark.read.json('/home/guilhermefmk/Área de Trabalho/WORKSPACE/Compass/Testes/a/', multiLine=True)

In [ ]:
teste = teste.withColumn("year", date_format(col("tweet_date"), "yyyy")) \
            .withColumn("month", date_format(col("tweet_date"), "MM")) \
            .withColumn("day", date_format(col("tweet_date"), "dd")) \
            .withColumn('tweet_text', regexp_replace('tweet_text','\n',' '))

In [ ]:
teste = teste.withColumn("tweet_date", date_format(col("tweet_date"), "yyyy-MM-dd hh:mm:ss"))

In [ ]:
teste = teste.withColumn("tweet_date",col("tweet_date").cast(TimestampType()))

In [ ]:
teste.where(teste.tweet_date.isNull()).show()

+---------+---+---+----------+----------+
|candidato|geo| id|tweet_date|tweet_text|
+---------+---+---+----------+----------+
+---------+---+---+----------+----------+



In [ ]:
teste.where(teste.candidato == 'Lula').show()

+---------+--------------------+-------------------+-------------------+--------------------+
|candidato|                 geo|                 id|         tweet_date|          tweet_text|
+---------+--------------------+-------------------+-------------------+--------------------+
|     Lula| Porto Velho, Brazil|1598293923822465024|2022-12-01 12:32:20|#ForaBolsonaroeSu...|
|     Lula|  Imperatriz, Brasil|1598293929551486976|2022-12-01 12:32:22|@JaspionRedPiLLL ...|
|     Lula|   Sao Paulo, Brazil|1598293932131385344|2022-12-01 12:32:22|@77_frota Pensei ...|
|     Lula|     Caucaia, Brasil|1598293937063542784|2022-12-01 12:32:24|@UOLNoticias Louc...|
|     Lula|Rio de Janeiro, B...|1598293951777509380|2022-12-01 12:32:27|@reporterenato @p...|
|     Lula| Porto Velho, Brazil|1598293977471496192|2022-12-01 12:32:33|@marcosdgb1 Bando...|
|     Lula|   Sao Paulo, Brazil|1598293988838371328|2022-12-01 12:32:36|@MarinaSilva http...|
|     Lula| Porto Velho, Brazil|1598294006441848833|2022-12-